# Sentiment Analysis
## The FAST AI way

In [1]:
from fastai.text import *
import html

In [2]:
PATH = Path("tsa/")

In [3]:
path_to_file = "twitter-airline-sentiment/"
file_name = "Tweets.csv"

LM_PATH = 'tsa/data/tweeter_lm'

CLS_PATH = 'tsa/data/tweeter_cls'

In [4]:
# Read csv
raw_data = pd.read_csv(path_to_file + file_name)

In [5]:
raw_data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
tweet_id                        14640 non-null int64
airline_sentiment               14640 non-null object
airline_sentiment_confidence    14640 non-null float64
negativereason                  9178 non-null object
negativereason_confidence       10522 non-null float64
airline                         14640 non-null object
airline_sentiment_gold          40 non-null object
name                            14640 non-null object
negativereason_gold             32 non-null object
retweet_count                   14640 non-null int64
text                            14640 non-null object
tweet_coord                     1019 non-null object
tweet_created                   14640 non-null object
tweet_location                  9907 non-null object
user_timezone                   9820 non-null object
dtypes: float64(2), int64(2), object(11)
memory usage: 1.7+ MB


In [7]:
# the goal if the challenge is to find classify a tweet sentiment based on text
# so we can discard the rest of the data
filtered_data  = raw_data[['airline_sentiment', 'text']]
filtered_data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [8]:
# convert sentiment text into numeric
def s_t_n(sentiment):
    if sentiment == "negative":
        return -1
    elif sentiment == "positive":
        return 1
    else:
        return 0

filtered_data['airline_sentiment'] = filtered_data['airline_sentiment'].apply(lambda x: s_t_n(x))
filtered_data.head()

/home/milla/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,airline_sentiment,text
0,0,@VirginAmerica What @dhepburn said.
1,1,@VirginAmerica plus you've added commercials t...
2,0,@VirginAmerica I didn't today... Must mean I n...
3,-1,@VirginAmerica it's really aggressive to blast...
4,-1,@VirginAmerica and it's a really big bad thing...


In [9]:
col_names=['labels', 'texts']

In [10]:
trn_text, val_text = sklearn.model_selection.train_test_split(
    filtered_data, test_size=0.1)

NameError: name 'sklearn' is not defined

In [11]:
text = filtered_data['text'].tolist()
df_trn = pd.DataFrame({'texts': text, 'labels':[0]*len(text)})

df_trn.to_csv(LM_PATH +'/train.csv', header='False', index=False)

In [12]:
df_trn.head()

,labels,texts
0,0,@VirginAmerica What @dhepburn said.
1,0,@VirginAmerica plus you've added commercials t...
2,0,@VirginAmerica I didn't today... Must mean I n...
3,0,@VirginAmerica it's really aggressive to blast...
4,0,@VirginAmerica and it's a really big bad thing...


In [13]:
# save the filted data
filtered_data.to_csv(CLS_PATH +'/train.csv', header='False', index=False)

## Language Model Tokens

In [14]:
chunksize=24000

In [15]:
BOS = 'xbos' # beginning of sentence tag
FLD = 'xfld' # data field tag

In [16]:
# plagerising from Jeremy my fixup replaces imoticons with something else
re1 = re.compile(r'  +')
def fixup(x):
    x = x.replace("@", " ").replace("http", "urlink")
    return re1.sub(' ', html.unescape(x))

In [17]:
def get_tokens(df, n_lbs=1):
    labels = df.iloc[:,range(n_lbs)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + '\n\n'.join(df[n_lbs].astype(str))
#     for i in range(n_lbs+1, len(df.columns)): texts += f' {FLD} {i-nlbs} ' +  df[i].astype(str)
#     texts.apply(fixup).values.astype(str)
    
    tok = Tokenizer().process_all(partition_by_cores(texts, 4))
    return tok, list(labels)

In [18]:
def get_all(df, n_lbls=1):
    tok, labels = [], []
    for i, r in enumerate(df):
        tok_, labels_ = get_tokens(r, n_lbls)
        tok += tok_
        labels += labels_
    return tok, labels

In [21]:
trn = pd.read_csv(LM_PATH + "/train.csv", header=None, chunksize=chunksize)
# trn = pd.read_csv(LM_PATH + "/train.csv", header=None)

In [22]:
tok, labels = get_all(trn)
# tok, labels = get_tokens(trn)

In [23]:
' '.join(tok[3])

'especially if xxup you switched my flight . \n \n  @usairways we bought our tickets months ago . xxmaj had seats together for all 6 . xxmaj have n\'t changed flight . xxmaj now 4 kids seats are scattered on plane . \n \n  @usairways 1 person to check in 50 people and the plane boards in 20 minutes and he is telling people @americanair exec bene \'s not honored \n \n  @usairways no problem ... just funny have a nice day \n \n  @usairways yes travel is complete flight 5095 from xxup clt to xxup cak xxmaj feb 18 \n \n  @usairways xxmaj cancelled xxmaj flightled due to maintenance . xxmaj the plane was there for 10 hours . xxmaj get your shit together because there are a lot of people unhappy . \n \n  @usairways a year and every time with xxup us xxmaj air something happens . i sat waiting for a re - scheduled flight for 10 hours then to say its \n \n  @usairways was switched to an xxmaj american xxmaj airlines flight , not by choice , and now i am charged $ 50 for bags ? xxmaj thought y\

In [24]:
np.save(path_to_file + 'lang_tokens.npy', tok)

In [25]:
tok = np.load(path_to_file + "lang_tokens.npy")
len(tok[0])

90738

In [26]:
freq = Counter(p for o in tok for p in o)
freq.most_common(25)

[('xxmaj', 22960),
 ('\n \n ', 14674),
 ('.', 13708),
 ('to', 8648),
 ('xxup', 7083),
 ('i', 6759),
 ('the', 6058),
 ('!', 4845),
 ('?', 4509),
 ('a', 4507),
 ('/', 4427),
 ('you', 4404),
 (',', 4145),
 ('for', 3996),
 ('flight', 3932),
 ('@united', 3861),
 ('on', 3810),
 ('and', 3730),
 ('#', 3651),
 ('my', 3286),
 ('@usairways', 2984),
 ('@americanair', 2939),
 ('is', 2933),
 ('in', 2579),
 ('@southwestair', 2440)]

In [27]:
max_vocab = 60000
min_freq = 2

In [28]:
itos = [o for o, c in freq.most_common(max_vocab) if c>min_freq]
itos.insert(0, '_unk_')
itos.insert(1, '_pad_')

In [29]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(stoi)

4707

In [47]:
trn_lm = np.array([[stoi[o] for o in p] for p in tok])

In [48]:
' '.join(str(o) for o in trn_lm[0])

'0 0 0 164 101 2 67 0 248 4 3 101 578 13 112 1182 2531 5 8 223 63 0 4 3 101 7 96 30 117 63 2 807 592 7 94 5 176 169 215 9 3 101 27 41 157 3765 5 0 0 61 1029 61 25 35 3178 314 3766 46 46 70 33 518 2786 3 101 19 27 41 11 157 491 237 500 102 27 3 101 458 86 305 119 317 11 16 15 219 34 96 30 33 44 2122 4 306 27 41 157 8 138 237 500 102 147 6 1985 3 101 191 14 1511 335 68 7 127 6 2304 44 236 0 0 233 242 652 151 481 297 3 101 2 157 307 11 3179 1777 15 2 3767 2 362 2 3768 0 14 83 4 593 32 12 12 49 12 0 3 101 2 258 14 7 0 717 48 6 57 7 6 43 9 6 0 3 101 27 36 381 14 19 594 58 111 377 4 2 13 154 199 158 5 31 4 3 101 96 13 134 34 2532 24 8 637 2123 808 29 1512 1778 2533 291 39 455 3 101 7 1443 718 0 4 62 204 240 186 0 0 4 6 2124 3 101 2 44 24 595 11 139 678 9 2 244 1513 102 21 658 215 5 0 46 46 7 33 30 161 979 18 21 382 215 272 9 462 2787 3 101 0 7 75 147 35 20 1986 20 2 0 1674 124 9 142 176 76 8 20 1987 481 51 194 53 32 12 12 49 12 0 3 101 2 55 9 3 101 6 331 39 6 1272 843 24 90 6 857 4 3 101 2 6

In [49]:
np.save(path_to_file + 'tok_ids.npy', trn_lm)
pickle.dump(itos, open(path_to_file + 'itos.pkl', 'wb'))

In [50]:
trn_lm = np.load(path_to_file + 'tok_ids.npy')
itos = pickle.load(open(path_to_file + 'itos.pkl', 'rb'))

In [34]:
# tok_lm = tok_lm[0] + tok_lm[1] + tok_lm[2] + tok_lm[3]

In [51]:
vs=len(itos)
vs,len(trn_lm)

(4707, 4)

In [52]:
len(trn_lm[2])

89653

# Wikitext103 conversion

In [37]:
em_sz, nh, nl = 400, 1150, 3

In [38]:
PRE_PATH = PATH/'models'/'wt103'
PRE_LM_PATH = PRE_PATH/'fwd_wt103.h5'

In [39]:
wgts = torch.load(PRE_LM_PATH, map_location=lambda storage, loc: storage)

In [40]:
enc_wgts = to_np(wgts['0.encoder.weight'])
row_m = enc_wgts.mean(0)

In [41]:
itos2 = pickle.load((PRE_PATH/'itos_wt103.pkl').open('rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

In [42]:
new_w = np.zeros((vs, em_sz), dtype=np.float32)
for i,w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r>=0 else row_m

In [43]:
wgts['0.encoder.weight'] = T(new_w)
wgts['0.encoder.with_droupout.embed.weight'] = T(np.copy(new_w))
wgts['1.decoder.weight'] = T(np.copy(new_w))

NameError: name 'T' is not defined

# Language Model

In [46]:
wd=1e-7
bptt=70
bs=52
opt_fn=partial(optim.Adam, betas=(0.8, 0.99))

In [54]:
t = len(np.concatenate(trn_lm))
t, t/64

(357623, 5587.859375)

In [57]:
trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
md = LanguageModelData(PATH, 1, vs, trn_dl, bs=bs,bppt=bppt)

NameError: name 'LanguageModelLoader' is not defined

In [58]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [60]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
                        dropouti=drops[0], droupout=drops[1], wdrop=drops[2],dropoute=drops[3],dropouth=drops[4])

learner.metrics = [accuracy]
leaner.unfreeze()

NameError: name 'md' is not defined

In [61]:
learner.get_layer_groups()

NameError: name 'learner' is not defined

In [62]:
learner.model.load_state_dict(wgts)

NameError: name 'leaner' is not defined

In [63]:
lr=1e-3
lrs = lr

In [64]:
learner.fit(lrs/2, 1, wds=wd, use_clr=(32), cycle_len=1)

NameError: name 'learner' is not defined

In [65]:
learner.save('lm_last_fit')

NameError: name 'learner' is not defined

In [66]:
learner.load('lm_last_fit')

NameError: name 'learner' is not defined

In [67]:
learner.unfreeze()

NameError: name 'learner' is not defined

In [68]:
learner.lr_find(srart_lr=lrs/10, end_lr=lrs*10, linear=True)

NameError: name 'learner' is not defined

In [69]:
learner.sched.plot()

NameError: name 'learner' is not defined

In [70]:
learner.fit(lrs, 1, wds=wd, use_clr=(20,10), cycle_len=15)

NameError: name 'learner' is not defined

In [71]:
learner.save('lm1')

NameError: name 'learner' is not defined

In [72]:
learner.save_encoder('lm1_enc')

NameError: name 'learner' is not defined

In [ ]:
learner.sched.plot()